In [ ]:
import pandas as pd
import glob
# 코드 테스트용 파일 생성
df = pd.read_csv('../../db/news_contents/news_contents_매일경제.csv')
test_df = df.head(100)
test_df.to_csv('test_sample.csv', index=False, encoding='utf-8-sig')

import os
file_path = '../../db/news_contents/news_contents_매일경제.csv'
filename = os.path.basename(file_path)
source_name = filename.split('_')[-1].replace('.csv', '')

In [21]:
!pip install ekonlpy

     ---------------------------------------- 0.0/33.2 MB ? eta -:--:--
     -- ------------------------------------- 2.1/33.2 MB 9.0 MB/s eta 0:00:04
     --------------- ----------------------- 12.8/33.2 MB 31.0 MB/s eta 0:00:01
     --------------------------- ----------- 23.1/33.2 MB 36.5 MB/s eta 0:00:01
     --------------------------------------  32.8/33.2 MB 38.5 MB/s eta 0:00:01
     ---------------------------------------- 33.2/33.2 MB 36.9 MB/s  0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/10.3 MB ? eta -:--:--
   -------------------------------------- - 10.0/10.3 MB 47.8 MB/s eta 0:00:01
   ---------------------------------------- 10.3/10.

  You can safely remove it manually.
  You can safely remove it manually.


In [8]:
import re

def clean_news(text):
    if not isinstance(text, str):
        return ''
    
    # HTML 태그 제거
    text = re.sub(r'<[^>]+>', '', text)
    
    # 기자 이메일, 언론사 이름 제거
    text = re.sub(r'\[.+?기자\]|\[.+?뉴스\]|\[.+?제공\]', '', text)
    text = re.sub(r'\/사진제공?=.+?$', '', text)
    text = re.sub(r'\/사진?=.+?$', '', text)
    
    # 홍보 문구 제거
    text = re.sub(r'이\s*기사는\s*.+?\s*에\s*게재된\s*기사입니다\.', '', text)

    # 줄바꿈 공백으로
    text = text.replace('\u2028', ' ').replace('\u2029', ' ').replace('\n', ' ')
    
    # 한글, 영어, 숫자, 공백, 마침표만 남기기 (마침표는 문장 구분을 위해 남겨야 함)
    text = re.sub(r'[^가-힣a-zA-Z0-9\s\.]', ' ', text)
    
    # 연속된 공백 하나로 줄이기
    text = ' '.join(text.split())
    
    return text

In [3]:
from ekonlpy.sentiment import MPCK

mpck = MPCK()

def get_ngram_tokens(text):
    if not text:
        return ""
    tokens = mpck.tokenize(text)
    ngrams = mpck.ngramize(tokens)
    final_result = tokens + ngrams    
    return ", ".join(final_result)


In [9]:
output_df = pd.DataFrame()
output_df['date'] = pd.to_datetime(test_df['date'])
output_df['date'] = output_df['date'].dt.date

output_df['content'] = test_df['content'].apply(clean_news)
output_df['tokens'] = output_df['content'].apply(get_ngram_tokens)
output_df['category'] = '뉴스'
output_df['source'] = source_name

In [10]:
output_df

,date,content,tokens,category,source
0,2025-11-14,14일 서울 중구 하나은행 본점 딜링룸 현황판에 코스피가 표시돼 있다. 나흘 연속 ...,"중구/NNG, 본점/NNG, 딜링룸/NNG, 현황/NNG, 판/NNG, 코스피/NN...",뉴스,매일경제
1,2025-10-21,NH농협은행이 수탁 중인 전국 광역자치단체 금고의 예치액에 대한 금리가 최대 1.2...,"수탁/NNG, 중/NNG, 전국/NNG, 금고/NNG, 예치/NNG, 액/NNG, ...",뉴스,매일경제
2,2025-12-18,유럽중앙은행 ECB 이 18일 현지시간 예금금리를 비롯한 3대 정책금리를 모두 동결...,"ecb/NNG, ecb/NNG, 현지/NNG, 예금/NNG, 금리/NNG, 비롯/N...",뉴스,매일경제
3,2025-12-14,은행채 금리상승에 조달비용 대형증권사 IMA 출시도 변수 증권사 머니무브 차단 목적...,"은행채/NNG, 금리/NNG, 상승/NNG, 조달/NNG, 비용/NNG, 출시/NN...",뉴스,매일경제
4,2025-11-22,비트코인 급락에 스트레티지도 60 폭락 MSCI 나스닥100등 벤치마크 제외가능성 ...,"비트코인/NNG, 급락/NNG, 하락/NNG, msci/NNG, 나스닥100/NNG...",뉴스,매일경제
...,...,...,...,...,...
95,2025-09-18,남은 10월 12월에도 인하할듯 올들어 첫 인하 0.25 P 인하 올해 GDP성장률...,"남/VV, 인하/NNG, 하/VV, 들/VV, 인하/NNG, 인하/NNG, gdp/...",뉴스,매일경제
96,2025-11-22,카뱅 3분기 최대 실적 경신 5대 지방은행 중 카뱅보다 누적 순익 많은 곳 부산은행...,"뱅/MAG, 최대/NNG, 실적/NNG, 경신/NNG, 지방은행/NNG, 중/NNG...",뉴스,매일경제
97,2025-10-21,6.27까지 임대차계약 맺고 이번에 규제지역 들어간 곳 퇴거자금 LTV 70 40 ...,"임대차/NNG, 계약/NNG, 맺/VV, 규제/NNG, 지역/NNG, 들어간/VV,...",뉴스,매일경제
98,2025-11-20,본 기사는 11월 20일 10 39 매일경제 자본시장 전문 유료매체인 레이더M 에 ...,"기사/NNG, 매일/NNG, 경제/NNG, 자본시장/NNG, 전문/NNG, 유료/N...",뉴스,매일경제
